# Large size CSV to DB

뭔가 한번에 csv를 DB table로 만들어주는 패키지가 있을 것 같았는데 없었다...
그래서 나름 이것저것 찾아보고 정리해보았다.
이게 최선인듯 하다.

다음과 같이 fun_CreateTable.py 함수 파일을 만들었다.
자세한 설명은 다음 링크에 해두었다.
>https://github.com/sugyeong-jo/Tech/blob/master/mysql/mysql_large%20size%20csv%20update%20to%20db.md



'''python
import csv
import ast
import pandas as pd
import time
from itertools import *



# data type 감지 함수
def dataType(val, current_type):
    try:
        # Evaluates numbers to an appropriate type, and strings an error
        t = ast.literal_eval(val)
    except ValueError:
        return 'varchar'
    except SyntaxError:
        return 'varchar'
    if type(t) in [int, float]:
        if (type(t) in [int]) and current_type not in ['float', 'varchar']:
            # Use smallest possible int type
            if (-32768 < t < 32767) and current_type not in ['int', 'bigint']:
                return 'smallint'
            elif (-2147483648 < t < 2147483647) and current_type not in ['bigint']:
                return 'int'
            else:
                return 'bigint'
        if type(t) is float and current_type not in ['varchar']:
            return 'decimal'
    else:
        return 'varchar'


# create Table sql 생성 함수
def createTable_stat(path, tablename):
    f = open(path, 'r')
    reader = csv.reader(f)
    longest, headers, type_list = [], [], []
    print('--------The data is followed--------')
    for row in islice(reader, 0, 5) :
        print(row)
        if len(headers) == 0:
            headers = row
            for col in row:
                longest.append(0)
                type_list.append('')
        else:
            for i in range(len(row)):
                # NA is the csv null value
                if type_list[i] == 'varchar' or row[i] == 'NA':
                    pass
                else:
                    var_type = dataType(row[i], type_list[i])
                    type_list[i] = var_type
            if len(row[i]) > longest[i]:
                longest[i] = len(row[i])
    f.close()
    print('------------------------------------')        

    statement = 'create table '+tablename+' ('

    for i in range(len(headers)):
        if type_list[i] == 'varchar':
            statement = (statement + '\n{} varchar({}),').format(headers[i].lower(), str(longest[i]))
        else:
            statement = (statement + '\n' + '{} {}' + ',').format(headers[i].lower(), type_list[i])

    statement = statement[:-1] + ');'
    print('--------The sql statement is followed--------')
    print(statement)
    print('---------------------------------------------')
    return statement
 
'''


In [3]:
#%%
from fun_CreateTable import *
import pymysql
#import csv
import ast
import pandas as pd
import time
import itertools
host_name = "localhost"
username = "sugyeong"
password = "12341234"
database_name = "kt_db"

db = pymysql.connect(
    host=host_name,  # DATABASE_HOST
    port=3306,
    user=username,  # DATABASE_USERNAME
    passwd=password,  # DATABASE_PASSWORD
    db=database_name,  # DATABASE_NAME
    charset='utf8'
)
cursor = db.cursor()

sql = """
set session 
sql_mode='STRICT_TRANS_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,NO_AUTO_CREATE_USER,NO_ENGINE_SUBSTITUTION';
"""
cursor.execute(sql)
db.commit()

# %%


다음은 데이터베이스 업로드 코드이다. 

In [13]:
#%%
################################################################################
######################## kt ####################################################
################################################################################

# CREATE TABLE
path='/var/lib/mysql-files/kt/ULSAN_NG_201801.csv'
sql=createTable_stat(path, 'etl_ymd')
cursor.execute(sql)
db.commit()


#%%
# INSERT
for i in range(1,13):
    month = format(i, '02')
    sql1 = "LOAD DATA INFILE '/var/lib/mysql-files/kt/ULSAN_NG_2018"+month+".csv"
    sql = sql1+"""
INTO TABLE kt_db.kt
FIELDS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
IGNORE 1 LINES;
    """
    print(sql)
    cursor.execute(sql)
    db.commit()




--------The data is followed--------
['id', 'x', 'y', 'timezn_cd', 'm00', 'm10', 'm15', 'm20', 'm25', 'm30', 'm35', 'm40', 'm45', 'm50', 'm55', 'm60', 'm65', 'm70', 'f00', 'f10', 'f15', 'f20', 'f25', 'f30', 'f35', 'f40', 'f45', 'f50', 'f55', 'f60', 'f65', 'f70', 'total', 'admi_cd', 'etl_ymd']
['100198947', '513202', '327093', '6', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '31140595', '20180119']
['100198947', '513202', '327093', '7', '0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '6', '31140595', '20180103']
['100198947', '513202', '327093', '8', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '6', '31140595', '20180102']
['100198947', '513202', '327093', '9', '0', '0', '0', '0', '0', '0', '1', '0', '1'

In [6]:
#%%
################################################################################
################### etl_ymd ####################################################
################################################################################

# CREATE TABLE
path='/var/lib/mysql-files/kt/etl_ymd.csv'
sql=createTable_stat(path, 'etl_ymd')
cursor.execute(sql)
db.commit()


#%%
# MODIFY
sql = """
ALTER TABLE etl_ymd MODIFY etl_ymd_datetype datetime;
"""
cursor.execute(sql)
db.commit()

# MODIFY
sql = """
ALTER TABLE etl_ymd MODIFY holiday_nm varchar(10);
"""
cursor.execute(sql)
db.commit()


#%%
# INSERT
sql = """
LOAD DATA INFILE '/var/lib/mysql-files/kt/etl_ymd.csv'
INTO TABLE kt_db.etl_ymd
FIELDS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;
"""
cursor.execute(sql)
db.commit()

#%%
# KEY
sql = "ALTER TABLE etl_ymd MODIFY COLUMN etl_ymd int(11) PRIMARY KEY"
cursor.execute(sql)
db.commit()


#%% 
# Check
sql = "SELECT * FROM etl_ymd;"
df = pd.read_sql(sql, db)
print(df)
df.columns




--------The data is followed--------
['etl_ymd', 'etl_ymd_dateType', 'weekday', 'holiday', 'holiday_nm']
['20180101', '2018-01-01', '0', '1', '신정']
['20180102', '2018-01-02', '1', '0', '0']
['20180103', '2018-01-03', '2', '0', '0']
['20180104', '2018-01-04', '3', '0', '0']
------------------------------------
--------The sql statement is followed--------
create table etl_ymd (
etl_ymd int,
etl_ymd_datetype varchar(0),
weekday smallint,
holiday smallint,
holiday_nm varchar(2));
---------------------------------------------
      etl_ymd etl_ymd_datetype  weekday  holiday holiday_nm
0    20180101       2018-01-01        0        1         신정
1    20180102       2018-01-02        1        0          0
2    20180103       2018-01-03        2        0          0
3    20180104       2018-01-04        3        0          0
4    20180105       2018-01-05        4        0          0
..        ...              ...      ...      ...        ...
360  20181227       2018-12-27        3        0    

Index(['etl_ymd', 'etl_ymd_datetype', 'weekday', 'holiday', 'holiday_nm'], dtype='object')

In [8]:
#%%
################################################################################
################### etl_ymd_weather ############################################
################################################################################

# CREATE TABLE
path='/var/lib/mysql-files/kt/etl_ymd_weather.csv'
sql=createTable_stat(path, 'etl_ymd_weather')
cursor.execute(sql)
db.commit()


#%%
# MODIFY
sql = """
ALTER TABLE etl_ymd_weather MODIFY etl_ymd_datetype datetime;
"""
cursor.execute(sql)
db.commit()

# MODIFY
sql = """
ALTER TABLE etl_ymd_weather MODIFY pm10_max varchar(10);
"""
cursor.execute(sql)
db.commit()

sql = """
ALTER TABLE etl_ymd_weather MODIFY pm10_min varchar(10);
"""
cursor.execute(sql)
db.commit()

sql = """
ALTER TABLE etl_ymd_weather MODIFY zungu_pm10_max varchar(10);
"""
cursor.execute(sql)
db.commit()

sql = """
ALTER TABLE etl_ymd_weather MODIFY zungu_pm10_min varchar(10);
"""
cursor.execute(sql)
db.commit()


#%%
# INSERT
sql = """
LOAD DATA INFILE '/var/lib/mysql-files/kt/etl_ymd_weather.csv'
INTO TABLE kt_db.etl_ymd_weather
FIELDS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
LINES TERMINATED BY '\r\n'
IGNORE 1 LINES;
"""
cursor.execute(sql)
db.commit()


#%%
# KEY
sql = "ALTER TABLE etl_ymd_weather MODIFY COLUMN etl_ymd int(11) PRIMARY KEY"
cursor.execute(sql)
db.commit()



#%% 
# Check
sql = "SELECT * FROM etl_ymd_weather;"
df = pd.read_sql(sql, db)
print(df)
df.columns



--------The data is followed--------
['etl_ymd', 'etl_ymd_dateType', 'weekday', 'max_temp', 'min_temp', 'pm10_max', 'pm10_min', 'zungu_pm10_max', 'zungu_pm10_min']
['20180101', '2018-01-01', '0', '5.5', '-0.1', '48', '24', '56', '25']
['20180102', '2018-01-02', '1', '7.6', '-0.4', '122', '34', '122', '37']
['20180103', '2018-01-03', '2', '4.6', '-1.8', '58', '9', '70', '17']
['20180104', '2018-01-04', '3', '5.4', '-2.9', '48', '13', '49', '20']
------------------------------------
--------The sql statement is followed--------
create table etl_ymd_weather (
etl_ymd int,
etl_ymd_datetype varchar(0),
weekday smallint,
max_temp decimal,
min_temp decimal,
pm10_max smallint,
pm10_min smallint,
zungu_pm10_max smallint,
zungu_pm10_min smallint);
---------------------------------------------
      etl_ymd etl_ymd_datetype  weekday  max_temp  min_temp pm10_max pm10_min  \
0    20180101       2018-01-01        0       6.0       0.0       48       24   
1    20180102       2018-01-02        1     

Index(['etl_ymd', 'etl_ymd_datetype', 'weekday', 'max_temp', 'min_temp',
       'pm10_max', 'pm10_min', 'zungu_pm10_max', 'zungu_pm10_min'],
      dtype='object')

In [9]:
#%%
###############################################
################### admi cd ###################
###############################################

# CREATE TABLE
path='/var/lib/mysql-files/kt/admi_cd.csv'
sql=createTable_stat(path, 'admi_cd')
cursor.execute(sql)
db.commit()


#%%
# MODIFY
sql = """
ALTER TABLE admi_cd MODIFY admi_nm varchar(10);
"""
cursor.execute(sql)
db.commit()


#%%
# INSERT
sql = """
LOAD DATA INFILE '/var/lib/mysql-files/kt/admi_cd.csv'
INTO TABLE kt_db.admi_cd
FIELDS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
IGNORE 1 LINES;
"""
cursor.execute(sql)
db.commit()


#%%
# KEY
sql = "ALTER TABLE admi_cd MODIFY COLUMN admi_cd int(11) PRIMARY KEY"
cursor.execute(sql)
db.commit()


#%% 
# Check
sql = "SELECT * FROM admi_cd;"
df = pd.read_sql(sql, db)
print(df)


--------The data is followed--------
['admi_cd', 'admi_nm']
['31140530', '신정3동']
['31140570', '삼산동']
['31140670', '야음장생포동']
['31140520', '신정2동']
------------------------------------
--------The sql statement is followed--------
create table admi_cd (
admi_cd int,
admi_nm varchar(6));
---------------------------------------------
     admi_cd admi_nm
0   31140510    신정1동
1   31140520    신정2동
2   31140530    신정3동
3   31140540    신정4동
4   31140550    신정5동
5   31140560      달동
6   31140570     삼산동
7   31140585     삼호동
8   31140595     무거동
9   31140600      옥동
10  31140625     대현동
11  31140635     수암동
12  31140640     선암동
13  31140670  야음장생포동


In [10]:
#%%
################################################################################
############################### id_WGS #########################################
################################################################################

# CREATE TABLE
path='/var/lib/mysql-files/kt/id_WGS.csv'
sql=createTable_stat(path, 'id_WGS')
cursor.execute(sql)
db.commit()


#%%
# MODIFY
sql = """
ALTER TABLE id_WGS MODIFY WGS_lat float(30,20);
"""
cursor.execute(sql)
db.commit()

sql = """
ALTER TABLE id_WGS MODIFY WGS_lon float(30,20);
"""
cursor.execute(sql)
db.commit()

#%%
# INSERT
sql = """
LOAD DATA INFILE '/var/lib/mysql-files/kt/id_WGS.csv'
INTO TABLE kt_db.id_WGS_rn
FIELDS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
IGNORE 1 LINES;
"""
cursor.execute(sql)
db.commit()

#%%
# KEY
sql = "ALTER TABLE id_WGS MODIFY COLUMN id int(11) PRIMARY KEY"
cursor.execute(sql)
db.commit()


#%% 
# Check
sql = "SELECT * FROM id_WGS;"
df = pd.read_sql(sql, db)
print(df)


--------The data is followed--------
['id', 'WGS_lat', 'WGS_lon']
['100198947', '35.53686026551888', '129.24617270595715']
['100198948', '35.53731085261797', '129.24617968347223']
['100211135', '35.53640397404951', '129.24671698782282']
['100211136', '35.53685456108953', '129.24672396826415']
------------------------------------
--------The sql statement is followed--------
create table id_WGS (
id int,
wgs_lat decimal,
wgs_lon decimal);
---------------------------------------------
              id    wgs_lat     wgs_lon
0      100198947  35.536861  129.246170
1      100198948  35.537312  129.246185
2      100211135  35.536404  129.246719
3      100211136  35.536854  129.246719
4      100211137  35.537304  129.246735
...          ...        ...         ...
25966  103392427  35.518158  129.390305
25967  103392428  35.518608  129.390305
25968  103392429  35.519058  129.390320
25969  103404575  35.499680  129.390533
25970  103404576  35.500130  129.390549

[25971 rows x 3 columns]


In [11]:
################################################################################
############################### id_RN #########################################
################################################################################

# CREATE TABLE
path='/var/lib/mysql-files/kt/id_RN.csv'
sql=createTable_stat(path, 'id_RN')
cursor.execute(sql)
db.commit()


#%%
# INSERT
sql = """
LOAD DATA INFILE '/var/lib/mysql-files/kt/id_RN.csv'
INTO TABLE kt_db.id_RN
FIELDS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
IGNORE 1 LINES;
"""
cursor.execute(sql)
db.commit()


#%%
# KEY
sql = "ALTER TABLE id_RN MODIFY COLUMN id int(11) PRIMARY KEY"
cursor.execute(sql)
db.commit()


#%% 
# Check
sql = "SELECT * FROM id_RN;"
df = pd.read_sql(sql, db)
df


--------The data is followed--------
['id', 'rn_cd']
['102892556', '3351512']
['102892557', '3351512']
['102892558', '3351512']
['102892560', '3351512']
------------------------------------
--------The sql statement is followed--------
create table id_RN (
id int,
rn_cd int);
---------------------------------------------


,id,rn_cd
0,100198947,1000026
1,100198948,1000026
2,100211135,1000026
3,100211136,1000026
4,100211137,1000026
...,...,...
25978,103392427,3011008
25979,103392428,3011008
25980,103392429,3011008
25981,103404575,4310749


In [12]:
# %%
################################################################################
############################### RN_geo #########################################
################################################################################

# CREATE TABLE
path = '/var/lib/mysql-files/kt/RN_geo.csv'
sql=createTable_stat(path, 'RN_geo')
cursor.execute(sql)
db.commit()


#%%
# MODIFY
sql = """
ALTER TABLE RN_geo MODIFY rn varchar(20);
"""
cursor.execute(sql)
db.commit()


#%%
# INSERT
sql = """
LOAD DATA INFILE '/var/lib/mysql-files/kt/RN_geo.csv'
INTO TABLE kt_db.RN_geo
FIELDS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
IGNORE 1 LINES;
"""
cursor.execute(sql)
db.commit()


#%%
# KEY
sql = "ALTER COLUMN geometry TYPE Geometry(LINESTRING, <SRID>) USING geometry(geom::Geometry, <SRID>)"
cursor.execute(sql)
db.commit()


#%% 
# Check
sql = "SELECT * FROM RN_geo;"
df = pd.read_sql(sql, db)
print(df)

--------The data is followed--------
['rn', 'rn_cd', 'geometry']
['울산고속도로', '1000008', '0    LINESTRING (129.2605590986902 35.5521783447833...\nName: geometry, dtype: object']
['동해고속도로', '1000026', '0    LINESTRING (129.2436992008299 35.5354969712652...\nName: geometry, dtype: object']
['산업로', '3000104', '0    LINESTRING (129.3226169976822 35.5080720161685...\nName: geometry, dtype: object']
['산업로', '3000104', '0    LINESTRING (129.3088430109262 35.5032402966481...\nName: geometry, dtype: object']
------------------------------------
--------The sql statement is followed--------
create table RN_geo (
rn varchar(0),
rn_cd int,
geometry varchar(84));
---------------------------------------------
              rn    rn_cd                                           geometry
0         울산고속도로  1000008  0    LINESTRING (129.2605590986902 35.55217834...
1         동해고속도로  1000026  0    LINESTRING (129.2436992008299 35.53549697...
2            산업로  3000104  0    LINESTRING (129.3226169976822 35.5